In [5]:
import numpy
import pandas
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.random import set_seed

tsla_data=pandas.read_csv('C:/Users/000110888/OneDrive - CSULB/Desktop/TSLA_Shocks.csv', 
index_col="Date", parse_dates=["Date"])

#splitting into training and testing sets
time_start = 2010
time_end = 2021
def train_test_split(time_start, time_end):
    train=tsla_data.loc[f"{time_start}":f"{time_end}", "Shock"].values
    test=tsla_data.loc[f"{time_end+1}":, "Shock"].values
    return train, test

train_set, test_set = train_test_split(time_start, time_end)

#splitting training data into samples
nsteps = 60

def split_sequence(sequence):
    x, y = list(), list()
    for i in range(len(sequence)):
        end_i = i + nsteps
        if end_i > len(sequence)-1:
            break
        seq_x, seq_y=sequence[i:end_i], sequence[end_i]
        x.append(seq_x)
        y.append(seq_y)
    return numpy.array(x), numpy.array(y)

train_x, train_y = split_sequence(train_set)

###############################################################
#FITTING LSTM MODEL
###############################################################
#reshaping train_x
features = 1
train_x=train_x.reshape(train_x.shape[0],train_x.shape[1],features)

#specifying model architecture
model_lstm = Sequential()
model_lstm.add(LSTM(units=6, activation="sigmoid", input_shape=(nsteps, features)))
model_lstm.add(Dense(units=1))

# Compiling the model
model_lstm.compile(loss="binary_crossentropy")
model_lstm.fit(train_x, train_y, epochs=5, batch_size=32)
inputs = tsla_data.loc[:,"Shock"][len(tsla_data.loc[:,"Shock"])-len(test_set)-nsteps :].values

#splitting into samples
test_x, test_y = split_sequence(inputs)

#reshaping
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], features)

#predicting for testing data
pred_prob=model_lstm.predict(test_x)

cutoff=[]
accuracy=[]
for i in range(99):
    tp=0
    tn=0
    cutoff.append(0.01*(i+1))
    for sub1, sub2 in zip(pred_prob, test_y):
        tp_ind=1 if (sub1>0.01*(i+1) and sub2==1) else 0
        tn_ind=1 if (sub1<0.01*(i+1) and sub2==0) else 0
        tp+=tp_ind
        tn+=tn_ind
    accuracy_i=(tp+tn)/len(pred_prob)
    accuracy.append(accuracy_i)
    
df=pandas.DataFrame({'accuracy': accuracy,'cut-off': cutoff})
max_accuracy=max(accuracy)
optimal=df[df['accuracy']==max_accuracy]
print(optimal)

###############################################################
#FITTING GRU MODEL
###############################################################
#specifying model architecture
model_gru = Sequential()
model_gru.add(GRU(units=6, activation="sigmoid", input_shape=(nsteps, features)))
model_gru.add(Dense(units=1))

# Compiling the model
model_gru.compile(loss="binary_crossentropy")
model_gru.fit(train_x, train_y, epochs=5, batch_size=32)

#predicting for testing data
pred_prob=model_lstm.predict(test_x)

cutoff=[]
accuracy=[]
for i in range(99):
    tp=0
    tn=0
    cutoff.append(0.01*(i+1))
    for sub1, sub2 in zip(pred_prob, test_y):
        tp_ind=1 if (sub1>0.01*(i+1) and sub2==1) else 0
        tn_ind=1 if (sub1<0.01*(i+1) and sub2==0) else 0
        tp+=tp_ind
        tn+=tn_ind
        
    accuracy_i=(tp+tn)/len(pred_prob)
    accuracy.append(accuracy_i)
    
df=pandas.DataFrame({'accuracy': accuracy,'cut-off': cutoff})
max_accuracy=max(accuracy)
optimal=df[df['accuracy']==max_accuracy]
print(optimal)


Epoch 1/5
89/89 [==============================] - 2s 10ms/step - loss: 0.7012
Epoch 2/5
89/89 [==============================] - 1s 10ms/step - loss: 0.6911
Epoch 3/5
89/89 [==============================] - 1s 12ms/step - loss: 0.6913
Epoch 4/5
89/89 [==============================] - 1s 11ms/step - loss: 0.6909
Epoch 5/5
10/10 [==============================] - 0s 3ms/step
    accuracy  cut-off
54  0.795302     0.55
55  0.795302     0.56
56  0.795302     0.57
57  0.795302     0.58
58  0.795302     0.59
59  0.795302     0.60
60  0.795302     0.61
61  0.795302     0.62
62  0.795302     0.63
63  0.795302     0.64
64  0.795302     0.65
65  0.795302     0.66
66  0.795302     0.67
67  0.795302     0.68
68  0.795302     0.69
69  0.795302     0.70
70  0.795302     0.71
71  0.795302     0.72
72  0.795302     0.73
73  0.795302     0.74
74  0.795302     0.75
75  0.795302     0.76
76  0.795302     0.77
77  0.795302     0.78
78  0.795302     0.79
79  0.795302     0.80
80  0.795302     0.81
81  0